In [ ]:
import pandas as pd
import numpy as np
import math
from geopy.distance import geodesic
from geopy.distance import distance
from surprise import Dataset, Reader, KNNBasic
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# 관광지 데이터
data = pd.read_csv('./model_test/merge_total.csv')
df = pd.DataFrame(data)
# 위도 : latitude / 경도 : longitude

# 사용자 데이터
u_data = pd.read_csv('./model_test/user_test.csv')
u_df = pd.DataFrame(u_data)

# 사용자 1명
user = u_df.iloc[[0]]

# 자연로그 변환
df['review_log'] = df.apply(lambda x: np.log(x['review']) if x['review'] > 0 else 0, axis=1)
df['rating_log'] = df.apply(lambda x: np.log(x['rating']) if x['rating'] > 0 else 0, axis=1)
df['price_log'] = df.apply(lambda x: np.log(x['mean_price']) if x['mean_price'] > 0 else 0, axis=1)

# 사용자의 위치 기반으로 직선거리 계산
# 사용자의 위치 정보
user_location = (user['latitude'].iloc[0], user['longitude'].iloc[0])

distances = []

for spot_index, spot_row in df.iterrows():
    # 관광지의 위치 정보
    spot_location = (spot_row['latitude'], spot_row['longitude'])
    
    # 두 지점 간의 거리 계산 (예시: km 단위)
    dist = distance(user_location, spot_location).km
    distances.append(dist)

# 거리 데이터를 데이터프레임에 추가
df['distance'] = distances

# minmax 정규화
def minmax_score(col):
    return (df[col] - df[col].min()) / (df[col].max() - df[col].min())
df['distance_score'] = minmax_score('distance')
df['review_score'] = minmax_score('review_log')
df['rating_score'] = minmax_score('rating_log')
df['price_score'] = minmax_score('price_log')

# 가중 점수 계산 = 가중치 * 특성
df['score'] = (
    user['rating_weight'][0] * df['rating_score'] +
    user['review_weight'][0] * df['review_score'] +
    user['price_weight'][0] * df['price_score'] +
    user['distance_weight'][0] * df['distance_score']
)

# 'score' 컬럼을 기준으로 정렬
sorted_df = df.sort_values(by='score', ascending=False)

# 상위 100개 행만 가진 새로운 데이터프레임 생성
top_100_df = sorted_df.head(100)

top_100_df